# Kütüphanelerin Yüklenmesi

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For m, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Veriyi yükleme

In [ ]:
df = pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")

# Veriye Genel Bakış

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.nunique()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

# Veri Görselleştirme

In [ ]:
df.iloc[:, 0:-1].hist(bins = 20, figsize = (20,30), color = "g",)
plt.show()

In [ ]:
con_1 = pd.melt(df,value_name='value', ignore_index=True)

# Seaborn customizations
sns.set_theme(style="darkgrid", font='serif', context='notebook')

# initializing our FacetGrid object and using map to create subplots
g = sns.FacetGrid(con_1, col='variable', col_wrap=3, sharex=False, sharey=False, height=4)
g = g.map(sns.distplot, 'value', color='r')

# Matplotlib customization
g.set_xticklabels(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df.corr(), annot=True, fmt=".2f", linecolor="w",linewidths=.5, cbar=False)
plt.show()

In [ ]:
np.abs(df.corr()["Class"]).sort_values(ascending = False)

In [ ]:
filtre = np.abs(df.corr()["Class"])>0.2
corr_columns = df.columns[filtre].tolist()
correlations = df[corr_columns].corr()

In [ ]:
plt.figure(figsize=(18,12))
sns.heatmap(correlations, annot = True, linewidths=.4, linecolor="w",fmt=".2f")
plt.show()

In [ ]:
df.drop(columns = ["Class"], axis = 1).corrwith(df.Class).plot.bar(figsize=(20,9), 
                                                 title="Correlation with Response Variable (Class)",
                                                 fontsize=15,
                                                 rot=45,
                                                 )
plt.grid(b=True, which='major', color='#666666', linestyle='-', alpha=.3)
plt.xticks(rotation=90)
plt.show()

In [ ]:
df_melted = pd.melt(df.iloc[:, 1:-1],  var_name='features', value_name='value')
plt.figure(figsize=(20,8))
sns.boxplot(x ='features', y='value', data=df_melted)
plt.xticks(rotation=90)
plt.show()

In [ ]:
df.Class.value_counts()

In [ ]:
data = [go.Histogram(x = df.Class)]
fig = go.Figure(data)
iplot(fig)

# Veri Ön İşleme

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()

In [ ]:
df["NormalizedAmount"] = sc.fit_transform(df["Amount"].values.reshape(-1, 1))

In [ ]:
df.drop(columns = ["Amount"], inplace = True)

In [ ]:
df.drop(columns = ["Time"], inplace = True)

In [ ]:
X = df.iloc[:, df.columns != "Class"]
y = df.iloc[:, df.columns == "Class"]

# Eğitim ve Test Verilerinin Hazırlanması

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test =  np.array(y_test)

# Model Seçilmesi ve Modelin Kurulması

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
model = Sequential([
    Dense(units = 16, input_dim = 29, activation = "relu"),
    Dense(units = 24, activation = "relu"),
    Dropout(0.5),
    Dense(20, activation = "relu"),
    Dense(24, activation = "relu"),
    Dense(1, activation = "sigmoid")
])

In [ ]:
model.summary()

# Modelin Eğitilmesi(Training)

## Yapay Sinir Ağları

In [ ]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
model.fit(X_train, y_train, batch_size = 15, epochs = 5)

In [ ]:
score = model.evaluate(X_test, y_test)

In [ ]:
print(score)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = np.rint(y_pred)

## Model Değerlendirmesi

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
f1_score(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators = 200)
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

## Model Değerlendirmesi

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
f1_score(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred)